In [55]:
import numpy as np
import matplotlib.pyplot as plt

# Cross Entropy Loss
def fCE(yhat, y):
    return -np.mean(np.log(yhat[y==1]))
# Percent Correct
def fPC(yhat, y):
    return np.mean(y.argmax(axis=1) == yhat.argmax(axis=1))

def predictor(w, X):
    z = np.exp(X.dot(w))
    z_sum = np.sum(z, axis=1).reshape(-1, 1)
    yhat = z/z_sum
    return yhat

def SGD(w, X, y):
    yhat = predictor(w, X)
    n = y.shape[0]
    w = X.T.dot(yhat-y)/n
    return w

# Given training and testing data, learning rate epsilon, and a specified batch size,
# conduct stochastic gradient descent (SGD) to optimize the weight matrix W (785x10).
# Then return W.
def softmaxRegression (trainingImages, trainingLabels, testingImages, testingLabels, epsilon = None, batchSize = None):
    n = trainingLabels.shape[0]
    W = 0.01 * np.random.randn(785,10)
    for n_tilde in range(n // batchSize):
        mini_x = trainingImages[n_tilde*batchSize: (n_tilde+1)*batchSize]
        mini_y = trainingLabels[n_tilde*batchSize: (n_tilde+1)*batchSize] 
        W = W - epsilon*SGD(W, mini_x, mini_y)
    print(fCE(predictor(W, testingImages), testingLabels))
    return W



In [56]:

# Load data
trainingImages = np.load("small_mnist_train_images.npy")
trainingLabels = np.load("small_mnist_train_labels.npy")
testingImages = np.load("small_mnist_test_images.npy")
testingLabels = np.load("small_mnist_test_labels.npy")


# Append a constant 1 term to each example to correspond to the bias terms
# ...
trainingImages = np.insert(trainingImages, 784, 1, axis=1)
testingImages = np.insert(testingImages, 784, 1, axis=1)
print(trainingImages[100: 200].shape)




(100, 785)


In [57]:
W = softmaxRegression(trainingImages, trainingLabels, testingImages, testingLabels, epsilon=0.1, batchSize=100)
    
# Visualize the vectors
# ...


TypeError: fCE() missing 1 required positional argument: 'y'

In [45]:
W

array([[-0.0033032 , -0.00129682,  0.00308001, ...,  0.0011795 ,
         0.00277746, -0.00353546],
       [ 0.01633821, -0.00052045,  0.00291887, ...,  0.00179568,
         0.00814619,  0.00665702],
       [-0.0117888 ,  0.01481778, -0.0007166 , ...,  0.00612499,
         0.00011741, -0.00410779],
       ...,
       [ 0.01549685,  0.01346133, -0.01054048, ...,  0.01105758,
         0.00886201, -0.00829809],
       [-0.00508249, -0.00537009,  0.01176527, ...,  0.00340047,
         0.00277077,  0.001106  ],
       [-0.04315699,  0.08371307, -0.03167513, ...,  0.03000081,
        -0.06556783, -0.02444724]])